####   Exemple

In [1]:
from Classes_Both.module_marche import DonneeMarche
from Classes_MonteCarlo_LSM.module_brownian import Brownian
from Classes_MonteCarlo_LSM.module_LSM import LSM_method
from Classes_Both.module_option import Option
from Classes_Both.derivatives import OptionDerivatives, OptionDerivativesParameters
import datetime as dt
import numpy as np
import time
from Classes_TrinomialTree.module_arbre_noeud import Arbre
import matplotlib.pyplot as plt
import pandas as pd

### Paramètre

In [2]:
start_date = dt.datetime(2025, 4, 4)
end_date = dt.datetime(2026, 4, 4)

market = DonneeMarche(date_debut= start_date,
volatilite=0.30, 
taux_interet=0.05, 
taux_actualisation=0.05,
# dividends=[{"ex_div_date": dt.datetime(2024, 4, 21), "amount": 3, "rate": 0}], 
dividende_ex_date = dt.datetime(2025, 1, 1),
dividende_montant = 0,
dividende_rate=0,
prix_spot=100)

optionA = Option(date_pricing=start_date, 
                maturite=end_date, 
                prix_exercice=100, call=False, americaine=True)

optionB = Option(date_pricing=start_date, 
                maturite=end_date, 
                prix_exercice=102, call=False, americaine=True)
# bermudien, 4 de chaque mois, donc mettre 1 pas de temps à chaque mois

optionC = Option(date_pricing=start_date, 
                maturite=end_date, # ici prix strike inultile, et type call put inutile
                prix_exercice=107, call=True, americaine=True)

pricer = LSM_method(optionC)

# nombre d'année entre début et fin 
period = (end_date - start_date).days / 365

nb_pas_arbre = 1000
arbre = Arbre(nb_pas_arbre, market, optionA, pruning = True)
arbre.pricer_arbre()

print(f"Prix arbre option {arbre.prix_option}")

Prix arbre option 9.867787904968338


### Test solo

In [3]:
nb_pas = 100 #12 #10000 #int((end_date - start_date).days / 2)*2
nb_chemin = 10000
seed = 1
method = 'vector'        # vector ou scalar
model_type="polynomial"  # "polynomial" "laguerre" "hermite" "legendre" "chebyshev"
degree_polynome = 2      # utile pour polynomial, laguerre, hermite, legendre, chebyshev

brownian = Brownian(period, nb_pas, nb_chemin, seed)
price, std_error, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=False,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price)
print("Ecart type: ", std_error)    


Nb chemins non antithetic: 10000
Prix min ('non antithetic', 'américaine', 'vector'): 0.7554609301551596
Prix max ('non antithetic', 'américaine', 'vector'): 0.7721395704730283
Prix:  0.7638002503140939
Ecart type:  0.0041696600794671815


### Itération nombre pas et chemin

In [ ]:
###### test boucle
from itertools import product
liste_chemin = [10000 * i for i in range(1, 11)]
liste_pas_chemin = [(10000 * i, int((end_date - start_date).days / 2)) for i in range(1, 11)]
liste_pas = [10 * i for i in range(1, 31)]
combinations = list(product(liste_chemin,liste_pas))
dico_price = {}

def test_boucle(combi):
    for (path,pas) in combi:
        brownian = Brownian(period, pas, path, 1)
        price, std_error, intervalle = pricer.LSM(brownian, market, method='vector', antithetic=True, 
                                                  poly_degree=2, model_type="polynomial",
                                                  print_info = False)
        if intervalle[0] <= arbre.prix_option <= intervalle[1]:
            if int(arbre.prix_option * 100) / 100 == int(price * 100) / 100:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4), 'proche': "✅    "}
                break
            if int(price * 100) / 100 - 0.01 <= int(arbre.prix_option * 100) / 100 <= int(price * 100) / 100 + 0.01:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4),'proche':'❌    '}
        print(pas,path, price)

test_boucle(combinations)
test_boucle(liste_pas_chemin)
pd.DataFrame(dico_price)

10 10000 14.526973161333691
20 10000 14.530328125307813
30 10000 14.75643479573864
40 10000 14.68480805766634
365 10000 14.617313205423969
365 20000 14.776423056455815
365 30000 14.724431473696399
365 40000 14.688987932695385
365 50000 14.698631369018662
365 60000 14.696266631366672
365 70000 14.683923758225264
365 80000 14.695121525090869
365 90000 14.704312852953846
365 100000 14.690874178371825


,50,365
,10000,30000
vrai prix,14.71668,14.71668
price,14.7126,14.7244
ecart-type,0.1083,0.0629
min,14.496,14.5986
max,14.9292,14.8503
proche,✅,❌


In [ ]:
nb_pas = int((end_date - start_date).days / 2)
nb_chemin = 10000
seed = 1
method = 'vector'        # vector ou scalar
model_type="polynomial"  # "polynomial" "laguerre" "hermite" "legendre" "chebyshev"
degree_polynome = 2      # utile pour polynomial, laguerre, hermite, legendre, chebyshev

# non antithetic
brownian = Brownian(period, nb_pas, nb_chemin, seed)
price, std_error_non_antithetic, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=False,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)

# antithetic
brownian = Brownian(period, nb_pas, nb_chemin, seed)
price_antithetic, std_error_antithetic, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=True,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price, std_error_non_antithetic)
print("Prix antithetic: ", price_antithetic, std_error_antithetic)
std_error_antithetic/std_error_non_antithetic - 1


Nb chemins non antithetic: 10000
Prix min ('non antithetic', 'américaine', 'vector'): 14.206988140219405
Prix max ('non antithetic', 'américaine', 'vector'): 15.024044644362519
Nb chemins antithetic: 10000
Prix min ('antithetic', 'américaine', 'vector'): 14.424295704137034
Prix max ('antithetic', 'américaine', 'vector'): 14.810330706710904
Prix:  14.615516392290962 0.20426412603577876
Prix antithetic:  14.617313205423969 0.09650875064346773


-0.5275296131707272

### Grecques

In [6]:
option_deriv = OptionDerivatives(optionC, market, pricer)  
print("Prix :", option_deriv.price(option_deriv.parameters))
print("Delta :", option_deriv.delta())
# print("Vega :", option_deriv.vega())
# print("Theta :", option_deriv.theta())
# print("Gamma :", option_deriv.gamma())


Prix : (0.7638002503140939, 0.0041696600794671815, (0.7554609301551596, 0.7721395704730283))
Delta : -0.01339760133817669


In [5]:
-0.4308289854187173*20

-8.616579708374346

In [5]:
-0.01339760133817669*1000

-13.39760133817669

In [ ]:
# prix C = 0.76
# delta C = 11